# Pandas Express
# TTC-Delays project
Team Members: 

Gaspare Castagna

Amirhossein Heidari

Avery Hoffer

Michael Metallo

In [1]:
import numpy as numpy
import pandas as pd
import os
import numpy as np
!pip install rapidfuzz
import re
import datetime as dt
import glob

Import, read, concat bus delay data

In [2]:
def read_excel_sheets(file_path):
    xls = pd.ExcelFile(file_path)
    data_frames = []
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name)
        data_frames.append(df)
    return pd.concat(data_frames, ignore_index=True)

In [23]:
df_2020 = read_excel_sheets('bus_2020.xlsx')

In [25]:
df_2020[df_2020['Delay'].isna()]

,Report Date,Route,Time,Day,Location,Incident,Delay,Gap,Direction,Vehicle,Date,Min Delay,Min Gap
360,2020-01-04,11.0,19:15:00,Saturday,Bayview and Eglinton,Diversion,NaN,NaN,N/B,8172.0,NaT,NaN,NaN
1784,2020-01-15,39.0,19:01:00,Wednesday,Finch and Middlefield,Diversion,NaN,NaN,WB,NaN,NaT,NaN,NaN
2030,2020-01-17,79.0,05:20:00,Friday,Pine and Lawrence,Late Leaving Garage - Operator,NaN,14.0,S/B,3725.0,NaT,NaN,NaN
2037,2020-01-17,924.0,05:26:00,Friday,Lawrence and Dufferin,Late Leaving Garage - Mechanical,NaN,9.0,S/B,9019.0,NaT,NaN,NaN
2038,2020-01-17,94.0,05:33:00,Friday,Ossington station,Late Leaving Garage - Mechanical,NaN,15.0,E/B,1116.0,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36146,NaT,36.0,23:44,Thursday,FINCH AND ISLINGTON,Security,NaN,NaN,W,3298.0,2020-12-31,10.0,20.0
36147,NaT,43.0,00:32,Thursday,KENNEDY STATION,Security,NaN,NaN,NaN,8706.0,2020-12-31,14.0,28.0
36148,NaT,86.0,01:08,Thursday,MEADOWVALE AND SHEPPAR,Emergency Services,NaN,NaN,E,8899.0,2020-12-31,10.0,20.0
36149,NaT,76.0,01:23,Thursday,LAKE SHORE AND MIMICO,Road Blocked - NON-TTC Collision,NaN,NaN,NaN,0.0,2020-12-31,86.0,101.0


In [3]:
df_2014 = read_excel_sheets('bus_2014.xlsx')
df_2015 = read_excel_sheets('bus_2015.xlsx') 
df_2016 = read_excel_sheets('bus_2016.xlsx') 
df_2017 = read_excel_sheets('bus_2017.xlsx') 
df_2018 = read_excel_sheets('bus_2018.xlsx')
df_2018 = df_2018.iloc[:, :-1]
df_2019 = read_excel_sheets('bus_2019.xlsx')
df_2019 = df_2019.iloc[:, :-3]
df_2020 = read_excel_sheets('bus_2020.xlsx')
df_2020 = df_2020.iloc[:, :-3]
df_2020.rename(columns={'Gap': 'Min Gap', 'Delay': 'Min Delay'}, inplace=True)
df_2021 = read_excel_sheets('bus_2021.xlsx')
df_2021.rename(columns={'Date': 'Report Date'}, inplace=True)
df_2021 = df_2021.iloc[:, :-3]
df_2022 = read_excel_sheets('bus_2022.xlsx')
df_2022.rename(columns={'Date': 'Report Date'}, inplace=True)

In [22]:
df_2020[df_2020['Min Delay'].isna()]

,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle
360,2020-01-04,11.0,19:15:00,Saturday,Bayview and Eglinton,Diversion,NaN,NaN,N/B,8172.0
1784,2020-01-15,39.0,19:01:00,Wednesday,Finch and Middlefield,Diversion,NaN,NaN,WB,NaN
2030,2020-01-17,79.0,05:20:00,Friday,Pine and Lawrence,Late Leaving Garage - Operator,NaN,14.0,S/B,3725.0
2037,2020-01-17,924.0,05:26:00,Friday,Lawrence and Dufferin,Late Leaving Garage - Mechanical,NaN,9.0,S/B,9019.0
2038,2020-01-17,94.0,05:33:00,Friday,Ossington station,Late Leaving Garage - Mechanical,NaN,15.0,E/B,1116.0
...,...,...,...,...,...,...,...,...,...,...
36146,NaT,36.0,23:44,Thursday,FINCH AND ISLINGTON,Security,NaN,NaN,W,3298.0
36147,NaT,43.0,00:32,Thursday,KENNEDY STATION,Security,NaN,NaN,NaN,8706.0
36148,NaT,86.0,01:08,Thursday,MEADOWVALE AND SHEPPAR,Emergency Services,NaN,NaN,E,8899.0
36149,NaT,76.0,01:23,Thursday,LAKE SHORE AND MIMICO,Road Blocked - NON-TTC Collision,NaN,NaN,NaN,0.0


In [4]:
batch1_list = [df_2014,df_2015,df_2016,df_2017,df_2018,df_2019,df_2020]
batch1 = pd.concat(batch1_list)

batch1['YTime'] = pd.to_datetime(batch1['Time'], format='%H:%M:%S', errors='coerce').dt.strftime('%H:%M')
batch1['DateTime'] = batch1['Report Date'].dt.strftime('%Y-%m-%d')+'-'+ batch1['YTime']
batch1['DateTime'] = pd.to_datetime(batch1['DateTime'])

drop_columns = ['YTime','Time','Report Date','Direction']
batch1 = batch1.drop(drop_columns, axis = 1)

batch2_list = [df_2021,df_2022]
batch2 = pd.concat(batch2_list)

batch2['Report Date'] = pd.to_datetime(batch2['Report Date'])
batch2['Time'] = pd.to_datetime(batch2['Time'], format='%H:%M').dt.time

# Create a new 'DateTime' column by combining 'Report Date' and 'Time'
batch2['DateTime'] = batch2.apply(
    lambda row: pd.to_datetime(row['Report Date'].replace(hour=row['Time'].hour, minute=row['Time'].minute, second=row['Time'].second)), axis=1
)

drop_columns = ['Time','Report Date','Direction']
batch2 = batch2.drop(drop_columns, axis = 1)
batch2 = batch2.dropna(subset=['Route'])

batches = [batch1, batch2]
df_bus_delay = pd.concat(batches)

In [20]:
batch2.head(1)

,Route,Day,Location,Incident,Min Delay,Min Gap,Vehicle,DateTime
0,320,Friday,YONGE AND STEELES,Emergency Services,0,0,1553,2021-01-01 04:26:00


In [5]:
df_bus_delay['copy'] = df_bus_delay['Location'].str.lower()
df_bus_delay['copy'] = df_bus_delay['copy'].str.replace('vp','victoria park')
df_bus_delay['copy'] = df_bus_delay['copy'].str.replace('b/w','').str.replace('w/b','').str.replace('e/b','').str.replace('s/b','').str.replace('n/b','')
df_bus_delay['copy'] = df_bus_delay['copy'].str.replace('stn','station').str.replace(' and ','&').str.replace('/','').str.replace(' ','')
df_bus_delay['copy'] = df_bus_delay['copy'].str.replace('(','').str.replace(')','')
df_bus_delay['copy'] = df_bus_delay['copy'].str.replace('@','&')
df_bus_delay['copy'] = df_bus_delay['copy'].astype(str)
df_bus_delay = df_bus_delay[~df_bus_delay['copy'].str.isdigit()]
df_bus_delay['copy'] = df_bus_delay['copy'].apply(lambda x: '&'.join(sorted(x.split('&'))) if x.count('&') == 1 else x)

stops = pd.read_csv('stops.txt')
stops['cleaned_stops'] = stops['stop_name'].str.replace('Side', '').str.replace(' ', '')
stops['cleaned_stops'] = stops['stop_name'].str.lower()
stops['cleaned_stops'] = stops['stop_name'].str.replace('(north)', '').str.replace('(west)', '').str.replace('(south)', '').str.replace('(east)', '')
stops['cleaned_stops'] = stops['cleaned_stops'].astype(str).str.replace('\d+', '', regex=True)


df_bus_delay_fuzz = df_bus_delay[~df_bus_delay['copy'].str.contains('route')].copy()
df_bus_delay_fuzz = df_bus_delay_fuzz[~df_bus_delay_fuzz['copy'].isin(stops['cleaned_stops'])]
df_bus_delay_fuzz = df_bus_delay_fuzz[df_bus_delay_fuzz['copy'].str.contains('&')]

unique_values = pd.unique(df_bus_delay_fuzz['copy'])

# Create a new DataFrame with a column of unique values
df_delay_fuzz = pd.DataFrame({
    'fuzz_locations': unique_values
})

df_bus_delay.to_csv('bus_delay_raw.csv')
df_delay_fuzz.to_csv('fuzz_locatiins.csv')



In the section below I am testing the fuzz score function we will be using:

This function takes in a string "location" and compares it to each entry in a list of strings "entry". 
It returns a string corresponding to the "best" match for the input string from the list of strings.
We can adjust the threshold to avoid false positives. Strings with no matches above the threshold will not be modified.

In [32]:
from rapidfuzz import process
def find_closest_match_threshold(row, reference_list, threshold = 40):
    match = process.extractOne(row, reference_list)
    if match[1] < threshold:
        return 'not able to fuzz'
    else:
        return match[0]

In [18]:
row = "yonge&steeles"
reference_list = ['Steeles & Yonge']
match = process.extractOne(row, reference_list)
print(match[1])

47.5


In [9]:
def find_closest_match_rapid(row, reference_list):
    match = process.extractOne(row, reference_list)
    return match[0]

In [33]:
df_delay_fuzz['fuzzed'] = df_delay_fuzz['fuzz_locations'].apply(find_closest_match_threshold, args=(stops['stop_name'],))

num_chunks = 10
chunk_size = len(df_delay_fuzz) // num_chunks

# Save each chunk into a separate CSV file
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size if i < num_chunks - 1 else len(df_delay_fuzz)
    
    chunk_df = df_delay_fuzz.iloc[start_idx:end_idx]
    
    # Save the chunk to a CSV file with a sequential number
    chunk_df.to_csv(f'df_delay_fuzz_{i + 1}.csv', index=False)

In [36]:
script_directory = os.path.dirname(os.path.abspath('df_delay_fuzz1.csv'))

# Use glob to find all .csv files in the directory containing 'bus_delay_fuzz'
csv_files = glob.glob(os.path.join(script_directory, '*df_delay_fuzz*.csv'))

# Initialize an empty list to store individual DataFrames
dfs = []

# Read and append each DataFrame to the list
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
df_fuzz = pd.concat(dfs, ignore_index=True)

In [38]:
bus_delays = pd.merge(df_bus_delay, df_fuzz, left_on='copy', right_on='fuzz_locations', how='left')

# Create the 'analysis' column and fill it with 'fuzz' if 'location' is not NaN, otherwise 'not fuzzed'
bus_delays['analysis'] = bus_delays['fuzzed'].where(~bus_delays['fuzz_locations'].isna(), bus_delays['copy'])

columns_to_drop = ['fuzz_locations', 'fuzzed','copy']
bus_delays = bus_delays.drop(columns=columns_to_drop)

In [39]:
bus_delays.head(50)

,Route,Day,Location,Incident,Min Delay,Min Gap,Vehicle,DateTime,analysis
0,95.0,Wednesday,York Mills station,Mechanical,10.0,20.0,1734.0,2014-01-01 00:23:00,yorkmillsstation
1,102.0,Wednesday,Entire run for route,General Delay,33.0,66.0,8110.0,2014-01-01 00:55:00,entirerunforroute
2,54.0,Wednesday,lawrence and Warden,Mechanical,10.0,20.0,7478.0,2014-01-01 01:28:00,Warden Ave at Lawrence Ave East
3,112.0,Wednesday,Kipling Station,Emergency Services,18.0,36.0,8084.0,2014-01-01 01:30:00,kiplingstation
4,24.0,Wednesday,VP and Ellesmere,Investigation,10.0,20.0,7843.0,2014-01-01 01:37:00,Ellesmere Rd at Victoria Park Ave
5,129.0,Wednesday,Scarborough Town,Mechanical,10.0,20.0,1755.0,2014-01-01 01:50:00,scarboroughtown
6,36.0,Wednesday,humberline at woodlot,Mechanical,16.0,32.0,1322.0,2014-01-01 02:28:00,humberlineatwoodlot
7,53.0,Wednesday,Passmore and Markham,Mechanical,18.0,36.0,7043.0,2014-01-01 02:30:00,Markham Rd at Passmore Ave
8,36.0,Wednesday,finch stn,Mechanical,16.0,30.0,7933.0,2014-01-01 02:33:00,finchstation
9,320.0,Wednesday,steele at yonge,Mechanical,7.0,14.0,8187.0,2014-01-01 02:44:00,steeleatyonge


In [40]:
num_chunks = 10
chunk_size = len(bus_delays) // num_chunks

# Save each chunk into a separate CSV file
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size if i < num_chunks - 1 else len(bus_delays)
    
    chunk_df = bus_delays.iloc[start_idx:end_idx]
    
    # Save the chunk to a CSV file with a sequential number
    chunk_df.to_csv(f'bus_delays_{i + 1}.csv', index=False)

In [ ]:
df_bus_delay.head(50)